# Pickling Game Objects

In this example, we demonstrate how to pickle game objects 
- using default implementation in `ggsolver`,
- how to modify default implementation.

Also applies to `ggsolver.parsers.automata`


In [1]:
import sys
sys.path.append('/home/ggsolver/')

## Default game objects

Most game objects in `ggsolver` can be constructed either in `explicit` or `symbolic` mode. In `explicit` mode, the game object stores a complete game graph as `networkx.MultiDiGraph`. On the other hand, in `symbolic` mode, game object stores predecessor, successor and transition functions that operate on `game.graph` that only remembers nodes to generate edges on-the-fly. 

Since pickle cannot save functions, the default implementation of game object pickling saves the source code of user-defined predecessor, successor and transition functions. 

Since these features apply by default to all `BaseGame` objects, game objects can be pickled as any other python objects. 

In [3]:
# Let's use deterministic two-player turn-based game for demo.
import ggsolver.dtptb as dtptb
import networkx as nx
import pickle 

# Create a game.
graph = nx.MultiDiGraph()
graph.add_nodes_from([(1, {"turn": 1}), (2, {"turn": 1}), (3, {"turn": 1})])
graph.add_edges_from([(1, 2, {"action": "a"}), (2, 3, {"action": "a"})])

game = dtptb.DtptbGame(name="test game")
game.construct_explicit(graph)

# Pickle game
game_str = pickle.dumps(game)

# Unpickle game
upkl_game = pickle.loads(game_str)

# Compare object states
print(f"game.__dict__:")
for k, v in game.__dict__.items():
    print(f"\t{k}: {v}")
print()
print(f"upkl_game.__dict__:")
for k, v in upkl_game.__dict__.items():
    print(f"\t{k}: {v}")

game.__dict__:
	_name: test game
	_graph: MultiDiGraph with 3 nodes and 2 edges
	_actions: {'a'}
	_delta: <function DtptbGame.construct_explicit.<locals>._delta at 0x7feb643319d8>
	_pred: <function DtptbGame.construct_explicit.<locals>._pred at 0x7feb64331268>
	_succ: <function DtptbGame.construct_explicit.<locals>._succ at 0x7feb64331048>
	_init_st: None
	_atoms: set()
	_label: None
	_properties: {}
	_is_constructed: True
	_mode: explicit
	_pkl_encode_func: ['_pred', '_succ', '_delta', '_label']
	_labeling_function: None

upkl_game.__dict__:
	_name: test game
	_graph: MultiDiGraph with 3 nodes and 2 edges
	_actions: {'a'}
	_delta: <function _delta at 0x7feb64331b70>
	_pred: <function _pred at 0x7feb64331840>
	_succ: <function _succ at 0x7feb64331ae8>
	_init_st: None
	_atoms: set()
	_label: None
	_properties: {}
	_is_constructed: True
	_mode: explicit
	_pkl_encode_func: ['_pred', '_succ', '_delta', '_label']
	_labeling_function: None


## Writing custom objects. 

By default the `_pred, _succ, _delta, _label` attributes in are pickled by fetching their source code while pickling and reconstructing the functions while unpickling. 

However, if a game object derived from `BaseGame` has other attributes that store functions that should be pickled, the class definition must extend `_pkl_encode_func` attribute of `BaseGame`. This attribute stores a list of game attributes that should be treated as function object for pickling process.

- Default: `_pkl_encode_func = ['_pred', '_succ', '_delta', '_label']`
- Extend as: `self._pkl_encode_func.extend([list of new attributes as strings])`

In [5]:
# Define a dummy function to use for demo.
def foo(a, b):
    return a + b

# Define a new class with _foo attribute that stores a function object. 
class NewGame(dtptb.DtptbGame):
    def __init__(self, name):
        super(NewGame, self).__init__(name)
        self._foo = foo
        self._pkl_encode_func.extend(['_foo'])

# Create NewGame object
game = NewGame('new game')
game.construct_explicit(graph)

# Pickle object
game_str = pickle.dumps(game)

# Unpickle game
upkl_game = pickle.loads(game_str)

# Compare object states
print(f"game.__dict__:")
for k, v in game.__dict__.items():
    print(f"\t{k}: {v}")
print()
print(f"upkl_game.__dict__:")
for k, v in upkl_game.__dict__.items():
    print(f"\t{k}: {v}")

game.__dict__:
	_name: new game
	_graph: MultiDiGraph with 3 nodes and 2 edges
	_actions: {'a'}
	_delta: <function DtptbGame.construct_explicit.<locals>._delta at 0x7feb64331d90>
	_pred: <function DtptbGame.construct_explicit.<locals>._pred at 0x7feb643316a8>
	_succ: <function DtptbGame.construct_explicit.<locals>._succ at 0x7feb64331c80>
	_init_st: None
	_atoms: set()
	_label: None
	_properties: {}
	_is_constructed: True
	_mode: explicit
	_pkl_encode_func: ['_pred', '_succ', '_delta', '_label', '_foo']
	_foo: <function foo at 0x7feb647662f0>
	_labeling_function: None

upkl_game.__dict__:
	_name: new game
	_graph: MultiDiGraph with 3 nodes and 2 edges
	_actions: {'a'}
	_delta: <function _delta at 0x7feb643312f0>
	_pred: <function _pred at 0x7feb64331400>
	_succ: <function _succ at 0x7feb64331f28>
	_init_st: None
	_atoms: set()
	_label: None
	_properties: {}
	_is_constructed: True
	_mode: explicit
	_pkl_encode_func: ['_pred', '_succ', '_delta', '_label', '_foo']
	_foo: <function foo at 

## Modifying `__setstate__` and `__getstate__`

This section provides pointers on how to override default pickling behavior. This should rarely be required, but is good to know. 

Two functions are important to modify python's default pickling behavior. 
* `__getstate__`: defines python object's state to be pickled. (https://docs.python.org/3/library/pickle.html#object.__getstate__)

* `__setstate__`: defines how python object's state is constructed given a dictionary of attribute names to python objects. (https://docs.python.org/3/library/pickle.html#object.__setstate__)
